##**TEAM:**
* Uma Maheshwari [C0928443]
* Jharana Adhikari [C0927442]
* Yasaswin Palukuri [C0928450]
* Raemil Corniel [C0922799]


##**Topic**

###Credit Card Fraud Detection

####**Dataset Details:**

Source: https://www.kaggle.com/datasets/mishra5001/credit-card/data

**I. About:**
- Get insights of the loan defaultes
- Get insights into customer spending behavior and loan defaulters.
- Identify patterns in credit usage, payments, and purchases.
- Perform customer segmentation to group similar spending profiles.
- Help financial institutions optimize marketing strategies and risk management.

**II. Problem statement**
- Financial institutions need to understand customer spending and repayment behavior to provide better credit offerings.
- The dataset helps in identifying potential defaulters by analyzing credit usage patterns.
- Dimensionality reduction (PCA) and clustering techniques can uncover hidden patterns in customer transactions.
- The goal is to segment customers for personalized financial products and risk assessment.

**III. Dataset Information**









### **Assignment**


Part I
1. Sample a subset of the data based on class label
2. Apply PCA and LDA and visualize the results
3. Perform classification using naïve Bayes and interpret the results
4. Perform classification using Logistic Regression and interpret the results
5. Perform classification using SVMs and interpret the results

6. Comment on which approaches might allow you to best identify the confidence of class assignments (e.g. how happy, or how sad rather than just a label)
7. Plot results based on the approach in f.

Part II
1. Apply PCA and LDA. For PCA, plot the incremental gain in capturing the variance in the data for each successive principal component that is added and compare this with the subset of data in step 1.
2. Perform classification using naïve Bayes and interpret and visualize the results
3. Perform classification using Logistic Regression and interpret and visualize the results
4. Perform classification using SVMs and interpret and visualize the results
5. Discuss which classes are most similar / different with some justification given
6. Explain how you might identify faces that are mislabeled in the dataset and provide an implementation.

* Optional

Based on a 7-dimensional vector of confidence values for face identity, plot the position of these in a 2D embedding. The “points” that are shown should be the actual face images within the plot.

